In [4]:
#pip install pymorphy2

     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 7.1MB 10.7MB/s 


In [5]:
#pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32917 sha256=98e1d075ca0ae39090d131bb750e9f6be1d80c002228b494a604a3baeb2b5b6e
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [6]:
import pandas as pd
import numpy as np

from string import punctuation
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import re
import nltk

In [7]:
max_words = 2000
max_len = 40

In [25]:
df = pd.read_excel('/content/отзывы за лето.xls')

df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [26]:
punct = set(punctuation)
stop_words_set = set(get_stop_words('ru') + get_stop_words('en'))
ma = MorphAnalyzer()

def preprocess(txt):
  r = str(txt).lower()
  r = ''.join(t for t in r if t not in punct)
  r = re.sub(r'не', 'не\s', r)
  r = re.sub(r'\snot', 'not', r)
  r = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9]+', ' ', r)
  # todo: надо бы убрать emoji

  r = " ".join([ma.parse(word)[0].normal_form for word in r.split() if word not in stop_words_set])
  r = r.strip()

  return r

In [27]:
df['Content'] = df['Content'].apply(preprocess)

In [28]:
df.drop(df[df['Content'] == ''].index, inplace=True)

In [29]:
df['target'] = df['Rating'] > 3
df['target'] = df['target'].astype('int')

In [30]:
df.head(20)

,Rating,Content,Date,target
0,5,just works,2017-08-14,1
1,4,целое удобноной приложениеиз минус хотеть боль...,2017-08-14,1
2,5,отлично,2017-08-14,1
3,5,зависать 1 работа антивирус рана sе пользовать...,2017-08-14,1
4,5,удобно работать быстро,2017-08-14,1
5,5,удобно норма,2017-08-14,1
6,5,удобный приложение,2017-08-14,1
7,5,устраивать,2017-08-14,1
8,5,работать чётко отличие банкомат вечно зависать...,2017-08-14,1
10,5,ок,2017-08-14,1


In [31]:
# иначе у нас не будет работать классификация
df['Rating'] -= 1

In [32]:
df['Rating'].unique()

array([4, 3, 1, 2, 0])

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
df_train, df_val = train_test_split(df, train_size=0.8)

In [35]:
df_train.head()

,Rating,Content,Date,target
15182,4,финансы рука,2017-07-22,1
20042,4,норма,2017-06-16,1
2407,4,s,2017-08-10,1
11664,4,удобно использование,2017-07-28,1
10746,4,довольный хороший приложение пользоваться удоб...,2017-07-30,1


In [36]:
df_val.head()

,Rating,Content,Date,target
8584,4,супер,2017-08-01,1
5571,4,обновление s мочь войти приложение отправить о...,2017-08-06,1
13340,4,нормальный работать,2017-07-25,1
9142,4,удобно один слово банк карман s,2017-08-01,1
12010,4,удобный управление,2017-07-28,1


In [38]:
#nltk.download("punkt")
train_tokens = nltk.word_tokenize(" ".join(df_train["Content"]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [39]:
tokens_filtered = [word for word in train_tokens if word.isalnum()]

dist = nltk.probability.FreqDist(tokens_filtered)
vocabulary = {pair[0]:i for i, pair in enumerate(dist.most_common(max_words-1), 1)}

In [40]:
tokens_filtered

['финансы',
 'рука',
 'норма',
 's',
 'удобно',
 'использование',
 'довольный',
 'хороший',
 'приложение',
 'пользоваться',
 'удобно',
 's',
 'терять',
 'поход',
 'банк',
 'приложение',
 'пропасть',
 'вклад',
 'довольный',
 'сброс',
 'настройка',
 'телефон',
 'заводский',
 's',
 'мочь',
 'скачать',
 'приложение',
 'глючить',
 'работать',
 'исправно',
 'довольный',
 'удобно',
 'простой',
 'супер',
 'спасиб',
 'сервиз',
 'простой',
 'удобный',
 'использование',
 'интерфейс',
 's',
 'супер',
 'говорят',
 'хакер',
 'взламывать',
 'мобильный',
 'банк',
 'тьфутьф',
 'мой',
 's',
 'добраться',
 'нормас',
 'новый',
 'прошивка',
 'sкорректно',
 'распознать',
 'root',
 'телефон',
 'meizu',
 'находить',
 'root',
 'sт',
 'просить',
 'скорректировать',
 'распознавание',
 'ос',
 'flyme',
 'заходить',
 'шаблон',
 'приложение',
 'вылетать',
 'шаблон',
 'почемуть',
 'отображаться',
 'контакт',
 'смартфон',
 'никак',
 's',
 'отличный',
 'приложение',
 'работать',
 'удобно',
 'быстро',
 'замечательный',


In [41]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = nltk.word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [42]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)

In [43]:
X_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 183,
       102], dtype=int32)

In [44]:
X_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

## Посмтроим модель на случайных весах

In [46]:
import keras
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPool1D, Activation, Embedding, Dense
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [47]:
num_classes = len(df_train["Rating"].unique())
y_train = keras.utils.to_categorical(df_train["Rating"], num_classes)
y_val = keras.utils.to_categorical(df_val["Rating"], num_classes)

In [48]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [49]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [50]:
early_stopping = EarlyStopping(monitor='val_loss')

In [51]:
epochs = 100
batch_size = 512

In [52]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/100
29/29 [==============================] - 1s 19ms/step - loss: 1.2523 - accuracy: 0.6781 - val_loss: 0.9403 - val_accuracy: 0.7146
Epoch 2/100
29/29 [==============================] - 0s 12ms/step - loss: 0.8418 - accuracy: 0.7291 - val_loss: 0.7723 - val_accuracy: 0.7578
Epoch 3/100
29/29 [==============================] - 0s 11ms/step - loss: 0.7202 - accuracy: 0.7643 - val_loss: 0.7024 - val_accuracy: 0.7635
Epoch 4/100
29/29 [==============================] - 0s 12ms/step - loss: 0.6587 - accuracy: 0.7796 - val_loss: 0.6669 - val_accuracy: 0.7685
Epoch 5/100
29/29 [==============================] - 0s 12ms/step - loss: 0.6223 - accuracy: 0.7864 - val_loss: 0.6563 - val_accuracy: 0.7779
Epoch 6/100
29/29 [==============================] - 0s 11ms/step - loss: 0.5954 - accuracy: 0.7956 - val_loss: 0.6524 - val_accuracy: 0.7729
Epoch 7/100
29/29 [==============================] - 0s 11ms/step - loss: 0.5788 - accuracy: 0.8011 - val_loss: 0.6523 - val_accuracy: 0.7735
Epoch 

In [53]:
score = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 16ms/step - loss: 0.6670 - accuracy: 0.7696
Test score: 0.6669819355010986
Test accuracy: 0.7696361541748047


Теперь поменяем цель с Rating на Target (с 5 классов на 2)

In [54]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["target"], num_classes)
y_val = keras.utils.to_categorical(df_val["target"], num_classes)

In [55]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [56]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [57]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/100
29/29 [==============================] - 1s 17ms/step - loss: 0.5550 - accuracy: 0.7841 - val_loss: 0.4489 - val_accuracy: 0.8068
Epoch 2/100
29/29 [==============================] - 0s 12ms/step - loss: 0.3798 - accuracy: 0.8277 - val_loss: 0.2733 - val_accuracy: 0.8833
Epoch 3/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2354 - accuracy: 0.9012 - val_loss: 0.2134 - val_accuracy: 0.9097
Epoch 4/100
29/29 [==============================] - 0s 12ms/step - loss: 0.1905 - accuracy: 0.9212 - val_loss: 0.2105 - val_accuracy: 0.9128
Epoch 5/100
29/29 [==============================] - 0s 12ms/step - loss: 0.1731 - accuracy: 0.9322 - val_loss: 0.2125 - val_accuracy: 0.9084


In [58]:
score = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 5ms/step - loss: 0.2319 - accuracy: 0.8976
Test score: 0.23191452026367188
Test accuracy: 0.8976160883903503


## Word2Vec
  
Теперь попробуем предобученные вектора

In [59]:
from gensim.models import Word2Vec

In [115]:
w2v = Word2Vec(size=128, window=5, min_count=2, workers=32)

In [116]:
def tokenize(text):
  tokens = nltk.word_tokenize(text)
  tokens_filtered = set([word for word in tokens if word.isalnum()])

  return tokens_filtered

In [117]:
ttokens = df_train['Content'].apply(tokenize)
ttokens

15182                                      {рука, финансы}
20042                                              {норма}
2407                                                   {s}
11664                              {использование, удобно}
10746    {s, пользоваться, удобно, приложение, банк, до...
                               ...                        
2056            {s, удобно, зане, sмаять, нравитце, лично}
10687                                             {удобно}
14028                          {войти, s, мочь, антивирус}
5090     {ограничить, перекладывать, сделать, разработа...
14803    {ошибка, mx5, счёт, синхронизировать, тип, кон...
Name: Content, Length: 15940, dtype: object

In [118]:
w2v.build_vocab(ttokens, progress_per=100)

In [119]:
w2v.train(ttokens, total_examples=w2v.corpus_count, epochs=20)

(1176687, 1668040)

In [120]:
embeddings = np.zeros((max_words, 128))

In [121]:
for word in vocabulary:
  if word in w2v.wv.vocab:
    embeddings[vocabulary[word]] = w2v.wv[word]

In [122]:
embeddings

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.1003058 ,  0.70635337,  0.3843517 , ...,  0.95103842,
        -0.14741552, -0.04479236],
       [ 0.19707786,  0.12353148,  0.5505985 , ...,  0.05843328,
         0.77665859,  0.42883533],
       ...,
       [ 0.19421297, -0.05573963,  0.27729911, ...,  0.32026148,
        -0.13680755,  0.15143128],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [123]:
num_classes = len(df_train["Rating"].unique())
y_train = keras.utils.to_categorical(df_train["Rating"], num_classes)
y_val = keras.utils.to_categorical(df_val["Rating"], num_classes)

In [125]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len, weights=[embeddings]))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [126]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [127]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/100
29/29 [==============================] - 1s 18ms/step - loss: 1.0365 - accuracy: 0.7041 - val_loss: 0.7312 - val_accuracy: 0.7622
Epoch 2/100
29/29 [==============================] - 0s 13ms/step - loss: 0.7195 - accuracy: 0.7592 - val_loss: 0.6808 - val_accuracy: 0.7654
Epoch 3/100
29/29 [==============================] - 0s 13ms/step - loss: 0.6814 - accuracy: 0.7654 - val_loss: 0.6669 - val_accuracy: 0.7716
Epoch 4/100
29/29 [==============================] - 0s 12ms/step - loss: 0.6597 - accuracy: 0.7684 - val_loss: 0.6639 - val_accuracy: 0.7704
Epoch 5/100
29/29 [==============================] - 0s 13ms/step - loss: 0.6452 - accuracy: 0.7717 - val_loss: 0.6596 - val_accuracy: 0.7685
Epoch 6/100
29/29 [==============================] - 0s 13ms/step - loss: 0.6327 - accuracy: 0.7733 - val_loss: 0.6567 - val_accuracy: 0.7654
Epoch 7/100
29/29 [==============================] - 0s 12ms/step - loss: 0.6267 - accuracy: 0.7740 - val_loss: 0.6567 - val_accuracy: 0.7666
Epoch 

In [128]:
score = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 5ms/step - loss: 0.6677 - accuracy: 0.7616
Test score: 0.6677179932594299
Test accuracy: 0.7616060376167297


Особой разницы в обучении и результатах не видно. Хотя, в теории, должно обучаться быстрее
  
Теперь попробуем с 2 классами

In [129]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["target"], num_classes)
y_val = keras.utils.to_categorical(df_val["target"], num_classes)

In [131]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len, weights=[embeddings]))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [132]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [133]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/100
29/29 [==============================] - 1s 19ms/step - loss: 0.3894 - accuracy: 0.8516 - val_loss: 0.2503 - val_accuracy: 0.8871
Epoch 2/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2430 - accuracy: 0.8941 - val_loss: 0.2284 - val_accuracy: 0.9015
Epoch 3/100
29/29 [==============================] - 0s 13ms/step - loss: 0.2240 - accuracy: 0.9009 - val_loss: 0.2212 - val_accuracy: 0.9009
Epoch 4/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2118 - accuracy: 0.9067 - val_loss: 0.2210 - val_accuracy: 0.8990
Epoch 5/100
29/29 [==============================] - 0s 12ms/step - loss: 0.2048 - accuracy: 0.9108 - val_loss: 0.2169 - val_accuracy: 0.9034
Epoch 6/100
29/29 [==============================] - 0s 12ms/step - loss: 0.1924 - accuracy: 0.9171 - val_loss: 0.2204 - val_accuracy: 0.9009


In [134]:
score = model.evaluate(X_val, y_val, batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 0s 5ms/step - loss: 0.2279 - accuracy: 0.8976
Test score: 0.22793227434158325
Test accuracy: 0.8976160883903503


Опять же нет разницы